In [226]:
import io
import random
import re
from collections import defaultdict

import numpy as np
import pandas as pd

In [147]:
ann_kor_file = './corpora/AnnaKarenina.txt'
war_pc_file = './corpora/WarAndPeace.txt'
war_pc_eng_file = './corpora/WarAndPeaceEng.txt'

In [148]:
def get_text(path):
    with io.open(path, encoding='utf-8') as file:
        return file.readlines()


text_ak = get_text(ann_kor_file)
text_wp = get_text(war_pc_file)

train_text = []
for line in text_wp + text_ak:
    cur_line = re.sub('[^а-я ]', '', line.lower())
    if cur_line.strip():
        train_text.append(cur_line)

random.shuffle(train_text)

In [150]:
RU_ALPHABET = 'АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя'
RU_ALPHABET = RU_ALPHABET[1::2] + ' '

In [151]:
TEST_SIZE = 5
TEST_LEN_SENT = 15

test_text = []
for ind, line in enumerate(train_text):
    if len(line) > TEST_LEN_SENT:
        test_text.append(line)
        train_text.pop(ind)
        if len(test_text) == TEST_SIZE:
            break


In [152]:
encoder_dict = dict(zip(RU_ALPHABET, random.sample(RU_ALPHABET, len(RU_ALPHABET))))

In [153]:
test_text_df = pd.DataFrame(list(zip(test_text, test_text.copy())), columns=['real', 'coded'])
test_text_df['coded'] = test_text_df['coded'].apply(lambda x: ''.join(encoder_dict[char] for char in x))
test_text_df.iloc[0]

real     князь андрей для полноты трофея пленников выст...
coded    музтэёиур цчёрызёбдыуджхёж дкцзёбыцуусмдпёпхйж...
Name: 0, dtype: object

In [154]:
def accuracy(df, decoded_col, real_col='real'):
    total_len = 0
    acc_len = 0
    for l_line, r_line in zip(df[decoded_col], df[real_col]):
        total_len += len(l_line)
        for l_ch, r_ch in zip(l_line, r_line):
            if l_ch == r_ch:
                acc_len += 1
    return acc_len / total_len

## Базовый частотный метод по Шерлоку Холмсу:

In [180]:
def get_frec_dict(text):
    frec_dict = defaultdict(int)
    for line in text:
        for char in line:
            frec_dict[char] += 1
    return frec_dict


frec_dict_train = get_frec_dict(train_text)
frec_dict_test = get_frec_dict(test_text_df['coded'])

In [321]:
def get_sort_keys(cur_dict):
    return dict(sorted(cur_dict.items(), key=lambda x: x[1], reverse=True)).keys()

In [181]:
frec_train_sort_key = get_sort_keys(frec_dict_train)
frec_test_sort_key = get_sort_keys(frec_dict_test)

decoder_dict = dict(zip(frec_test_sort_key, frec_train_sort_key))

In [183]:
for ind, line in enumerate(test_text_df['coded']):
    test_text_df.at[ind, 'decoded'] = ''.join(decoder_dict[char] for char in line)

In [184]:
accuracy(test_text_df, decoded_col='decoded')

0.5147420147420148

In [193]:
test_text_df.iloc[0]['real']

'князь андрей для полноты трофея пленников выставленный также вперед на глаза императору не мог не привлечь его внимания наполеон видимо вспомнил что он видел его на поле и обращаясь к нему употребил то самое наименование молодого человека    под которым болконский в первый раз отразился в его памяти'

In [194]:
test_text_df.iloc[0]['decoded']

'уипчб аидвеж дсп мосиоть твоюеп мсеиинуол льрталсеииьж тауйе лмевед иа гсача нкмеватовя ие ког ие мвнлсеыб его линкаинп иамосеои лнднко лрмокинс ыто ои лндес его иа мосе н озваэапрб у иекя ямотвезнс то ракое ианкеиолаине косодого ыесолеуа    мод уотовьк зосуоирунж л мевльж вач отвачнсрп л его макптн'

## Частотный метод для биграмм

In [379]:
def get_frec_dict_ngram(text, ngram=2):
    frec_dict = defaultdict(int)
    for line in text:
        for ind, _ in enumerate(line[:len(line) - ngram + 1]):
            frec_dict[line[ind:ind + ngram]] += 1
    return frec_dict


frec_dict_train_bigram = get_frec_dict_ngram(train_text)
frec_dict_test_bigram = get_frec_dict_ngram(test_text_df['coded'])

In [201]:
frec_train_sort_key_bigram = get_sort_keys(frec_dict_train_bigram)
frec_test_sort_key_bigram = get_sort_keys(frec_dict_test_bigram)

decoder_dict_bigram = dict(zip(frec_test_sort_key_bigram, frec_train_sort_key_bigram))

In [207]:
def decode_text(df_text, decod_dict, frec_dict_coded, ngram=2):
    for ind, line in enumerate(df_text['coded']):
        if len(line) < ngram:
            df_text.at[ind, f'decoded_{ngram}'] = line
        else:
            decoded_line = []
            for char_ind, _ in enumerate(line):
                l_ngram = line[char_ind - ngram + 1:char_ind + 1] if char_ind - ngram + 1 >= 0 else None
                r_ngram = line[char_ind:char_ind + ngram] if char_ind + ngram <= len(line) else None

                if not l_ngram:
                    decoded_line.append(decod_dict[r_ngram])
                    continue

                if l_ngram and r_ngram:
                    if frec_dict_coded[l_ngram] >= frec_dict_coded[r_ngram]:
                        decoded_line.append(decod_dict[r_ngram][1:])
                    else:
                        decoded_line[-1] = '' if len(decoded_line[-1]) == 1 else decoded_line[-1][:1]
                        decoded_line.append(decod_dict[r_ngram])
            df_text.at[ind, f'decoded_{ngram}'] = ''.join(decoded_line)

In [208]:
decode_text(test_text_df, decoder_dict_bigram, frec_dict_test_bigram)

In [212]:
accuracy(test_text_df, decoded_col='decoded_2')

0.1511056511056511

In [213]:
test_text_df.iloc[0]['real']

'князь андрей для полноты трофея пленников выставленный также вперед на глаза императору не мог не привлечь его внимания наполеон видимо вспомнил что он видел его на поле и обращаясь к нему употребил то самое наименование молодого человека    под которым болконский в первый раз отразился в его памяти'

In [214]:
test_text_df.iloc[0]['decoded_2']

'певд д зо чо нсно отйкабалоесьно  тр  воое н п б трнеоальух е оь чс и в рер вв оь етор  и су н и о лиж тк нини        н ил оттлае алм и  с оа  горти оае алегоини и о от са отнечдоу ра итк ме оаочтнготи кос  силм  й о    сустмкони рттроел в  о осракторслобствлаылуое о ь аноннерпоанерн оне нини  оседо'

## MCMC-сэмплирование

In [419]:
def get_likelihood(text, decoder, ngram_dict, coef, ngram=2 ):
    likelihood = 0
    decoded_line = ''.join(decoder[char] for char in text)
    for ind, _ in enumerate(decoded_line[:len(decoded_line) - ngram + 1]):
        likelihood += np.log(ngram_dict.get(decoded_line[ind: ind + ngram], 1/coef))
    return likelihood

In [420]:
def random_change_el(keys_list):
    l_key, r_key = random.sample(range(len(keys_list)), k=2)
    new_list = keys_list.copy()
    new_list[r_key], new_list[l_key] = new_list[l_key], new_list[r_key]
    return new_list

In [421]:
def is_update_likelihood(old_l, new_l):
    if new_l > old_l:
        return True
    else:
        return random.random() < np.exp(new_l - old_l)

In [493]:
def train(train_t, test, iter=15000, tryed=5, ngram=2):
#tryed - для решения проблемы с локальными минимумами
    dict_frec_train_prob = get_frec_dict_ngram(train_t, ngram=ngram)
    coef_count = sum(dict_frec_train_prob.values())
    for key in dict_frec_train_prob.keys():
        dict_frec_train_prob[key] /= coef_count

    best_likelihood = -np.inf
    best_decoder = {}

    for _ in range(tryed):
        frec_train = list(get_sort_keys(get_frec_dict(train_t)))
        frec_test = list(get_sort_keys(get_frec_dict(test)))
        decoder_dict = dict(zip(frec_test, frec_train))
        likelihood = get_likelihood(''.join(test), decoder_dict, dict_frec_train_prob, coef_count, ngram=ngram)
        for _ in range(iter):
            new_frec_train = random_change_el(frec_train)
            new_decoder_dict = dict(zip(frec_test, new_frec_train))
            new_likelihood = get_likelihood(''.join(test), new_decoder_dict, dict_frec_train_prob, coef_count, ngram=ngram)
            if is_update_likelihood(likelihood, new_likelihood):
                likelihood = new_likelihood
                frec_train = new_frec_train

                if new_likelihood > best_likelihood:
                    best_likelihood = new_likelihood
                    best_decoder = new_decoder_dict
    return best_likelihood, best_decoder


In [446]:
likelihood_mcmc, decoder_mcmc = train(train_text, test_text_df['coded'])

In [447]:
likelihood_mcmc

-4467.754185450096

In [448]:
for ind, line in enumerate(test_text_df['coded']):
    test_text_df.at[ind, 'decoded_mcmc'] = ''.join(decoder_mcmc[char] for char in line)

In [449]:
accuracy(test_text_df, 'decoded_mcmc')

0.9963144963144963

In [450]:
test_text_df.iloc[0]['real']

'князь андрей для полноты трофея пленников выставленный также вперед на глаза императору не мог не привлечь его внимания наполеон видимо вспомнил что он видел его на поле и обращаясь к нему употребил то самое наименование молодого человека    под которым болконский в первый раз отразился в его памяти'

In [451]:
test_text_df.iloc[0]['decoded_mcmc']

'князь андрей для полноты троцея пленников выставленный также вперед на глаза императору не мог не привлечь его внимания наполеон видимо вспомнил что он видел его на поле и обращаясь к нему употребил то самое наименование молодого человека    под которым болконский в первый раз отразился в его памяти'

## Тестовое задание

In [477]:
main_test = """←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"""

In [494]:
main_likelihood, main_decoder = train(train_text, main_test)

In [495]:
main_likelihood

-1242.95892914957

In [496]:
main_decoded_line = ''.join(main_decoder[char] for char in main_test)
main_decoded_line

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'

### Попробуем увеличить количество символов в n-грамме

In [433]:
real_test = "если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю"

In [436]:
main_likelihood_3, main_decoder_3 = train(train_text, main_test, ngram=3)
main_decoded_line_3 = ''.join(main_decoder_3[char] for char in main_test)
main_likelihood_3

-1737.1210389749433

In [437]:
main_decoded_line_3

'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю'

In [438]:
def calc_acc (real, decoded):
    acc = 0
    for couple_char in zip(real, decoded):
        if couple_char[0] == couple_char[1]:
            acc += 1
    return acc/len(real)

In [439]:
calc_acc(real_test, main_decoded_line_3)

1.0

## Применение

Думаю, такой способ неплохо покажет себя как базовая модель перевода с одного языка на другой (если ничего неизвестно про язык)